# Restaurant Performance & Menu Optimization Analysis

## Project Overview

This project analyzes a quarter’s worth of transactional order data from a fictitious restaurant serving international cuisine to evaluate menu performance, revenue drivers, and operational efficiency. The goal is to move beyond descriptive insights and deliver clear, data-backed recommendations that improve profitability and operational decision-making.

Using transaction-level order data, this analysis identifies underperforming menu items, high-value orders, peak and off-peak demand periods, and cuisine-level opportunities for menu optimization and staffing alignment.

## Business Problem

Restaurants often face margin pressure due to menu bloat, inefficient staffing, and misalignment between customer demand and operational resources. Without data-driven insights, decisions around menu changes and staffing levels are typically based on intuition rather than evidence.

This project addresses the following core questions:

Which menu items and categories truly drive value?

Where are resources being allocated inefficiently?

How can menu and staffing decisions be optimized to improve profitability?

## Data Overview
The dataset used in this project is sourced from the [Maven Analytics Restaurant Orders Dataset](https://mavenanalytics.io/data-playground/restaurant-orders), representing a fictitious restaurant and designed for analytical practice.

It consists of three tables:
- **order_details**: Transaction-level order records including order ID, date, time, and item ID  
- **menu_items**: Menu metadata including item name, category, and price  
- **order_analysis (view)**: A derived analytical view joining orders and menu data with calculated revenue and standardized time fields  

Records with missing `item_id` values were excluded from analytical views to ensure accurate item-level revenue analysis, while remaining available in the raw tables for auditability.

## Project Objectives

1. Identify high- and low-performing menu items based on order volume and revenue contribution.

2. Analyze category- and cuisine-level performance to uncover demand patterns.

3. Examine high-spend orders to understand purchasing behavior and revenue concentration.

4. Evaluate temporal ordering trends to identify peak and off-peak periods.

5. Assess operational efficiency by aligning order volume with staffing demand.

6. Provide actionable recommendations for menu optimization and staffing adjustments.

In [1]:
#Import necessary modules
import pandas as pd
import os
import sqlite3

In [2]:
# Connect to the SQLite database
db_path = os.path.join(os.getcwd(), 'restaurant_performance.db')
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


In [3]:
# Create tables and load data from CSV files
# Create menu_items table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS menu_items (
        menu_item_id INTEGER PRIMARY KEY,
        item_name TEXT,
        category TEXT,
        price REAL
    )
""")

# Create order_details table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS order_details (
        order_details_id INTEGER PRIMARY KEY,
        order_id INTEGER,
        order_date DATE,
        order_time TIME,
        item_id INTEGER
    )
""")

# Load menu_items from CSV
menu_df = pd.read_csv('Restaurant Orders/menu_items.csv')
menu_df.to_sql('menu_items', conn, if_exists='replace', index=False)

# Load order_details from CSV
order_details_df = pd.read_csv('Restaurant Orders/order_details.csv')
order_details_df.to_sql('order_details', conn, if_exists='replace', index=False)

conn.commit()
print("✓ Tables created and data loaded successfully!")

✓ Tables created and data loaded successfully!


In [4]:
# Verify data loaded correctly
cursor.execute("SELECT COUNT(*) FROM order_details")
order_count = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM menu_items")
menu_count = cursor.fetchone()[0]

cursor.execute("SELECT MIN(order_date), MAX(order_date) FROM order_details")
date_range = cursor.fetchone()

print(f"Order Details Records: {order_count:,}")
print(f"Menu Items Records: {menu_count}")
print(f"Date Range: {date_range[0]} to {date_range[1]}")

Order Details Records: 12,234
Menu Items Records: 32
Date Range: 1/1/23 to 3/9/23


In [5]:
# Menu Items performance overview
menu_df = pd.read_sql_query("SELECT * FROM menu_items", conn)

print("Menu Items Summary:")
print(f"Total unique menu items: {len(menu_df)}")
print(f"\nMenu Items by Category:")
print(menu_df['category'].value_counts())
print(f"\nPrice Range: ${menu_df['price'].min()} - ${menu_df['price'].max()}")
print(f"\nMenu items with missing data:")
print(menu_df[menu_df.isnull().any(axis=1)])

Menu Items Summary:
Total unique menu items: 32

Menu Items by Category:
category
Mexican     9
Italian     9
Asian       8
American    6
Name: count, dtype: int64

Price Range: $5.0 - $19.95

Menu items with missing data:
Empty DataFrame
Columns: [menu_item_id, item_name, category, price]
Index: []


- The menu is evenly distributed across four cuisines, with Mexican and Italian offering the largest item variety.
- Menu prices range from $5.00 to $19.95, allowing for both high- and low-ticket order behavior.

In [6]:
# Check for missing values and data quality issues
print("=" * 60)
print("MISSING VALUES ANALYSIS")
print("=" * 60)

# Missing values in order_details
cursor.execute("""
    SELECT 
        COUNT(*) as total_records,
        COUNT(CASE WHEN item_id IS NULL THEN 1 END) as null_item_ids,
        COUNT(CASE WHEN order_date IS NULL THEN 1 END) as null_dates,
        COUNT(CASE WHEN order_time IS NULL THEN 1 END) as null_times
    FROM order_details
""")
result = cursor.fetchone()
print(f"\norder_details table:")
print(f"  Total records: {result[0]:,}")
print(f"  NULL item_ids: {result[1]} ({result[1]/result[0]*100:.2f}%)")
print(f"  NULL order_dates: {result[2]}")
print(f"  NULL order_times: {result[3]}")


MISSING VALUES ANALYSIS

order_details table:
  Total records: 12,234
  NULL item_ids: 137 (1.12%)
  NULL order_dates: 0
  NULL order_times: 0


In [7]:

# Missing values in menu_items
cursor.execute("""
    SELECT 
        COUNT(*) as total_items,
        COUNT(CASE WHEN item_name IS NULL THEN 1 END) as null_names,
        COUNT(CASE WHEN category IS NULL THEN 1 END) as null_categories,
        COUNT(CASE WHEN price IS NULL THEN 1 END) as null_prices
    FROM menu_items
""")
result = cursor.fetchone()
print(f"\nmenu_items table:")
print(f"  Total items: {result[0]}")
print(f"  NULL item_names: {result[1]}")
print(f"  NULL categories: {result[2]}")
print(f"  NULL prices: {result[3]}")


menu_items table:
  Total items: 32
  NULL item_names: 0
  NULL categories: 0
  NULL prices: 0


Initial data profiling revealed that approximately 1.12% of order records contain missing item_id values. Because these records cannot be reliably linked to menu items, they are excluded from item-level revenue and performance analysis. The raw records are retained in the source table for completeness and auditability, while downstream analysis is conducted on a cleaned analytical view.

In [8]:
# Check for invalid data - prices, duplicates, unmatched items
print("\n" + "=" * 60)
print("DATA QUALITY CHECKS")
print("=" * 60)

# Price validation
cursor.execute("SELECT COUNT(*), MIN(price), MAX(price) FROM menu_items")
count, min_price, max_price = cursor.fetchone()
print(f"\nPrice range (menu_items): ${min_price:.2f} - ${max_price:.2f}")



DATA QUALITY CHECKS

Price range (menu_items): $5.00 - $19.95


In [9]:

# Check for items in orders that don't exist in menu_items
cursor.execute("""
    SELECT COUNT(*) as unmatched_items
    FROM order_details od
    WHERE od.item_id NOT IN (SELECT menu_item_id FROM menu_items)
    AND od.item_id IS NOT NULL
""")
unmatched = cursor.fetchone()[0]
print(f"Orders referencing non-existent items: {unmatched}")


Orders referencing non-existent items: 0


In [10]:

# Duplicate check
cursor.execute("SELECT COUNT(*), COUNT(DISTINCT order_details_id) FROM order_details")
total, distinct = cursor.fetchone()
print(f"Duplicate order_details_ids: {total - distinct}")

cursor.execute("SELECT COUNT(*), COUNT(DISTINCT menu_item_id) FROM menu_items")
total, distinct = cursor.fetchone()
print(f"Duplicate menu_item_ids: {total - distinct}")


Duplicate order_details_ids: 0
Duplicate menu_item_ids: 0


In [11]:

# Sample of records with NULL item_ids
print("\nSample records with NULL item_ids:")
cursor.execute("SELECT order_details_id, order_id, order_date FROM order_details WHERE item_id IS NULL LIMIT 5")
for row in cursor.fetchall():
    print(f"  - order_details_id: {row[0]}, order_id: {row[1]}, date: {row[2]}")


Sample records with NULL item_ids:
  - order_details_id: 122, order_id: 50, date: 1/1/23
  - order_details_id: 298, order_id: 125, date: 1/2/23
  - order_details_id: 358, order_id: 147, date: 1/3/23
  - order_details_id: 387, order_id: 161, date: 1/3/23
  - order_details_id: 470, order_id: 200, date: 1/3/23


In [12]:
# Summary statistics before cleaning
print("\n" + "=" * 60)
print("SUMMARY STATISTICS")
print("=" * 60)

# Unique orders
cursor.execute("SELECT COUNT(DISTINCT order_id) FROM order_details")
unique_orders = cursor.fetchone()[0]
print(f"\nUnique orders: {unique_orders:,}")


# Items per order
cursor.execute("""
    SELECT 
        AVG(items_per_order) as avg_items,
        MIN(items_per_order) as min_items,
        MAX(items_per_order) as max_items
    FROM (
        SELECT COUNT(*) as items_per_order
        FROM order_details
        WHERE item_id IS NOT NULL
        GROUP BY order_id
    )
""")
avg_items, min_items, max_items = cursor.fetchone()
print(f"Items per order: avg={avg_items:.2f}, min={min_items}, max={max_items}")


# Menu categories
cursor.execute("SELECT COUNT(DISTINCT category) FROM menu_items")
num_categories = cursor.fetchone()[0]
print(f"Menu categories: {num_categories}")

cursor.execute("""
    SELECT category, COUNT(*) as item_count
    FROM menu_items
    GROUP BY category
    ORDER BY item_count DESC
""")
print("\nItems by category:")
for row in cursor.fetchall():
    print(f"  - {row[0]}: {row[1]} items")


SUMMARY STATISTICS

Unique orders: 5,370
Items per order: avg=2.26, min=1, max=14
Menu categories: 4

Items by category:
  - Mexican: 9 items
  - Italian: 9 items
  - Asian: 8 items
  - American: 6 items


- Rather than modifying or deleting raw records, data quality issues were addressed by creating a cleaned analytical view. Records with missing item references were excluded from downstream analysis, while all source tables were retained unchanged to preserve data integrity and auditability.

In [13]:
cursor.execute("DROP VIEW IF EXISTS order_analysis;")
conn.commit()


In [14]:

cursor.execute("""
CREATE VIEW order_analysis AS
SELECT
    od.order_id,
    DATE(od.order_date) AS order_date,
    od.order_time AS raw_order_time,

    CASE
        WHEN od.order_time LIKE '%AM'
             AND CAST(SUBSTR(od.order_time, 1, INSTR(od.order_time, ':') - 1) AS INTEGER) = 12
            THEN 0
        WHEN od.order_time LIKE '%AM'
            THEN CAST(SUBSTR(od.order_time, 1, INSTR(od.order_time, ':') - 1) AS INTEGER)
        WHEN od.order_time LIKE '%PM'
             AND CAST(SUBSTR(od.order_time, 1, INSTR(od.order_time, ':') - 1) AS INTEGER) = 12
            THEN 12
        ELSE
            CAST(SUBSTR(od.order_time, 1, INSTR(od.order_time, ':') - 1) AS INTEGER) + 12
    END AS order_hour,

    od.item_id,
    mi.item_name,
    mi.category,
    mi.price,
    mi.price AS revenue
FROM order_details od
JOIN menu_items mi
    ON od.item_id = mi.menu_item_id
WHERE od.item_id IS NOT NULL
  AND od.order_time IS NOT NULL;
""")
conn.commit()


In [18]:
# Check order_analysis table
print("=" * 60)
print("ORDER ANALYSIS TABLE")
print("=" * 60)

# Count records
cursor.execute("SELECT COUNT(*) FROM order_analysis") 
count = cursor.fetchone()[0] 
print(f"\nTotal records in order_analysis: {count:,}")

 # Display first 10 records
print("\nFirst 10 records:") 
cursor.execute("SELECT * FROM order_analysis LIMIT 10") 
columns = [description[0] for description in cursor.description] 
print(f"\nColumns: {', '.join(columns)}\n") 
print(f"\nColumns: {', '.join(columns)}\n")

for row in cursor.fetchall():
    print(row)

# Check for NULL item_ids
cursor.execute("SELECT COUNT(*) FROM order_analysis WHERE item_id IS NULL")
null_count = cursor.fetchone()[0]
print(f"\nNULL item_ids: {null_count}")

# Verify item names are populated
cursor.execute("SELECT COUNT(*) FROM order_analysis WHERE item_name IS NOT NULL")
named_items = cursor.fetchone()[0]
print(f"Records with item names: {named_items:,}")

# Verify revenue is calculated
cursor.execute("SELECT COUNT(*) FROM order_analysis WHERE revenue IS NOT NULL")
revenue_count = cursor.fetchone()[0]
print(f"Records with revenue calculated: {revenue_count:,}")


ORDER ANALYSIS TABLE

Total records in order_analysis: 12,097

First 10 records:

Columns: order_id, order_date, raw_order_time, order_hour, item_id, item_name, category, price, revenue


Columns: order_id, order_date, raw_order_time, order_hour, item_id, item_name, category, price, revenue

(1, None, '11:38:36 AM', 11, 109.0, 'Korean Beef Bowl', 'Asian', 17.95, 17.95)
(2, None, '11:57:40 AM', 11, 108.0, 'Tofu Pad Thai', 'Asian', 14.5, 14.5)
(2, None, '11:57:40 AM', 11, 124.0, 'Spaghetti', 'Italian', 14.5, 14.5)
(2, None, '11:57:40 AM', 11, 117.0, 'Chicken Burrito', 'Mexican', 12.95, 12.95)
(2, None, '11:57:40 AM', 11, 129.0, 'Mushroom Ravioli', 'Italian', 15.5, 15.5)
(2, None, '11:57:40 AM', 11, 106.0, 'French Fries', 'American', 7.0, 7.0)
(3, None, '12:12:28 PM', 12, 117.0, 'Chicken Burrito', 'Mexican', 12.95, 12.95)
(3, None, '12:12:28 PM', 12, 119.0, 'Chicken Torta', 'Mexican', 11.95, 11.95)
(4, None, '12:16:31 PM', 12, 117.0, 'Chicken Burrito', 'Mexican', 12.95, 12.95)
(5, None, '

- All subsequent analysis is conducted at the order–item level, where each row represents a single menu item purchased within an order.

### Menu Performance

In [19]:
#Top 15 Revenue Generators
print("\n" + "=" * 80)
print("Top 15 HIGHEST-REVENUE ITEMS")
print("=" * 80)
pd.read_sql_query("""
SELECT
    item_name,
    category,
    COUNT(*) AS times_ordered,
    ROUND(SUM(revenue), 2) AS total_revenue,
    ROUND(SUM(revenue) / COUNT(*), 2) AS avg_price_per_order
FROM order_analysis
GROUP BY item_name, category
ORDER BY total_revenue DESC
LIMIT 15
""", conn)



Top 15 HIGHEST-REVENUE ITEMS


,item_name,category,times_ordered,total_revenue,avg_price_per_order
0,Korean Beef Bowl,Asian,588,10554.60,17.95
1,Spaghetti & Meatballs,Italian,470,8436.50,17.95
2,Tofu Pad Thai,Asian,562,8149.00,14.50
3,Cheeseburger,American,583,8132.85,13.95
4,Hamburger,American,622,8054.90,12.95
5,Orange Chicken,Asian,456,7524.00,16.50
6,Eggplant Parmesan,Italian,420,7119.00,16.95
7,Steak Torta,Mexican,489,6821.55,13.95
8,Chicken Parmesan,Italian,364,6533.80,17.95
9,Pork Ramen,Asian,360,6462.00,17.95


In [20]:
# Bottom 10 Worst Performers
print("\n" + "=" * 80)
print("BOTTOM 10 LOWEST-REVENUE ITEMS")
print("=" * 80)

pd.read_sql_query("""
SELECT
    item_name,
    category,
    COUNT(*) AS times_ordered,
    ROUND(SUM(revenue), 2) AS total_revenue,
    ROUND(SUM(revenue) / COUNT(*), 2) AS avg_price_per_order
FROM order_analysis
GROUP BY item_name, category
ORDER BY total_revenue ASC
LIMIT 10
               """, conn)




BOTTOM 10 LOWEST-REVENUE ITEMS


,item_name,category,times_ordered,total_revenue,avg_price_per_order
0,Chicken Tacos,Mexican,123,1469.85,11.95
1,Potstickers,Asian,205,1845.00,9.00
2,Chips & Guacamole,Mexican,237,2133.00,9.00
3,Hot Dog,American,257,2313.00,9.00
4,Cheese Quesadillas,Mexican,233,2446.50,10.50
5,Veggie Burger,American,238,2499.00,10.50
6,Steak Tacos,Mexican,214,2985.30,13.95
7,Edamame,Asian,620,3100.00,5.00
8,Cheese Lasagna,Italian,207,3208.50,15.50
9,Chips & Salsa,Mexican,461,3227.00,7.00


In [21]:
#Revenue Concentration
print("\n" + "=" * 80)
print("REVENUE CONCENTRATION BY ITEM")
print("=" * 80)

pd.read_sql_query("""
SELECT
    item_name,
    ROUND(SUM(revenue), 2) AS total_revenue,
    ROUND(
        SUM(revenue) * 100.0 / 
        (SELECT SUM(revenue) FROM order_analysis),
        2
    ) AS revenue_percentage
FROM order_analysis
GROUP BY item_name
ORDER BY total_revenue DESC;
""", conn)



REVENUE CONCENTRATION BY ITEM


,item_name,total_revenue,revenue_percentage
0,Korean Beef Bowl,10554.60,6.63
1,Spaghetti & Meatballs,8436.50,5.30
2,Tofu Pad Thai,8149.00,5.12
3,Cheeseburger,8132.85,5.11
4,Hamburger,8054.90,5.06
5,Orange Chicken,7524.00,4.73
6,Eggplant Parmesan,7119.00,4.47
7,Steak Torta,6821.55,4.28
8,Chicken Parmesan,6533.80,4.10
9,Pork Ramen,6462.00,4.06


In [22]:
#Revenue Concentration by Orders
print("\n" + "=" * 40)
print("REVENUE CONCENTRATION BY Orders")
print("=" * 40)

pd.read_sql_query("""
SELECT
    item_name,
    COUNT(*) AS times_ordered,
    ROUND(SUM(revenue), 2) AS total_revenue
FROM order_analysis
GROUP BY item_name
ORDER BY times_ordered DESC;
""", conn)



REVENUE CONCENTRATION BY Orders


,item_name,times_ordered,total_revenue
0,Hamburger,622,8054.90
1,Edamame,620,3100.00
2,Korean Beef Bowl,588,10554.60
3,Cheeseburger,583,8132.85
4,French Fries,571,3997.00
5,Tofu Pad Thai,562,8149.00
6,Steak Torta,489,6821.55
7,Spaghetti & Meatballs,470,8436.50
8,Mac & Cheese,463,3241.00
9,Chips & Salsa,461,3227.00


- Top item (Korean Beef Bowl) alone generates 6.63% of total revenue.

- Top 10 items account for ~43% of total revenue.

- Indicates strong dependency on a small subset of menu items.

### Category Performance

- Which categories drive the most revenue?

In [23]:
#Revenue Concentration by Category
print("\n" + "=" * 40)
print("REVENUE CONCENTRATION BY CATEGORY")
print("=" * 40)

pd.read_sql_query("""
SELECT
    category,
    COUNT(*) AS items_sold,
    ROUND(SUM(revenue), 2) AS total_revenue,
    ROUND(AVG(revenue), 2) AS avg_item_price
FROM order_analysis
GROUP BY category
ORDER BY total_revenue DESC;
""", conn)



REVENUE CONCENTRATION BY CATEGORY


,category,items_sold,total_revenue,avg_item_price
0,Italian,2948,49462.70,16.78
1,Asian,3470,46720.65,13.46
2,Mexican,2945,34796.80,11.82
3,American,2734,28237.75,10.33


#### Category Revenue Share

In [24]:
# Revenue Share by Category
print("\n" + "=" * 40)
print("REVENUE SHARE BY CATEGORY")
print("=" * 40)

pd.read_sql_query("""
SELECT
    category,
    ROUND(SUM(revenue), 2) AS total_revenue,
    ROUND(
        SUM(revenue) * 100.0 / 
        (SELECT SUM(revenue) FROM order_analysis),
        2
    ) AS revenue_share_pct
FROM order_analysis
GROUP BY category
ORDER BY total_revenue DESC;
""", conn)



REVENUE SHARE BY CATEGORY


,category,total_revenue,revenue_share_pct
0,Italian,49462.70,31.07
1,Asian,46720.65,29.34
2,Mexican,34796.80,21.85
3,American,28237.75,17.74


#### Item Variety vs Revenue

In [25]:
# Item Diversity vs Revenue by Category
print("\n" + "=" * 40)
print("ITEM DIVERSITY VS REVENUE BY CATEGORY")
print("=" * 40)

pd.read_sql_query("""
SELECT
    category,
    COUNT(DISTINCT item_name) AS menu_items,
    ROUND(SUM(revenue), 2) AS total_revenue
FROM order_analysis
GROUP BY category
ORDER BY total_revenue DESC;
""", conn)



ITEM DIVERSITY VS REVENUE BY CATEGORY


,category,menu_items,total_revenue
0,Italian,9,49462.70
1,Asian,8,46720.65
2,Mexican,9,34796.80
3,American,6,28237.75


- Italian: 31.07% of total revenue

-  Asian: 29.34%, despite higher item volume

- Italian’s higher average price ($16.78) drives revenue dominance.

In [26]:
# Ordes that Generate the Most Revenue
print("\n" + "=" * 40)
print("ORDERS THAT GENERATE THE MOST REVENUE")
print("=" * 40)

pd.read_sql_query("""
SELECT
    order_id,
    ROUND(SUM(revenue), 2) AS order_total
FROM order_analysis
GROUP BY order_id
ORDER BY order_total DESC
LIMIT 10;
""", conn)



ORDERS THAT GENERATE THE MOST REVENUE


,order_id,order_total
0,440,192.15
1,2075,191.05
2,1957,190.10
3,330,189.70
4,2675,185.10
5,4482,184.50
6,1274,183.55
7,2188,182.65
8,3473,182.55
9,3583,179.60


In [27]:
# What Items were sold in the Top 10 Highest Revenue Orders?
print("\n" + "=" * 60)
print("ITEMS SOLD IN THE TOP 10 HIGHEST REVENUE ORDERS")
print("=" * 60)

pd.read_sql_query("""
SELECT
    order_id,
    item_name,
    category,
    revenue
FROM order_analysis
WHERE order_id IN (
    SELECT order_id
    FROM order_analysis
    GROUP BY order_id
    ORDER BY SUM(revenue) DESC
    LIMIT 10
)
ORDER BY order_id, revenue DESC;
""", conn)



ITEMS SOLD IN THE TOP 10 HIGHEST REVENUE ORDERS


,order_id,item_name,category,revenue
0,330,Spaghetti & Meatballs,Italian,17.95
1,330,Korean Beef Bowl,Asian,17.95
2,330,Chicken Parmesan,Italian,17.95
3,330,Orange Chicken,Asian,16.50
4,330,Salmon Roll,Asian,14.95
...,...,...,...,...
130,4482,Steak Torta,Mexican,13.95
131,4482,Chicken Burrito,Mexican,12.95
132,4482,Chicken Burrito,Mexican,12.95
133,4482,California Roll,Asian,11.95


In [28]:
# Low Frequency Items (Less than 300 orders)
print("\n" + "=" * 60)
print("LOW FREQUENCY ITEMS (LESS THAN 300 ORDERS)")
print("=" * 60)
pd.read_sql_query("""
SELECT
    item_name,
    COUNT(*) AS times_ordered,
    ROUND(SUM(revenue), 2) AS total_revenue
FROM order_analysis
GROUP BY item_name
HAVING COUNT(*) < 300
ORDER BY total_revenue ASC;
""", conn)



LOW FREQUENCY ITEMS (LESS THAN 300 ORDERS)


,item_name,times_ordered,total_revenue
0,Chicken Tacos,123,1469.85
1,Potstickers,205,1845.00
2,Chips & Guacamole,237,2133.00
3,Hot Dog,257,2313.00
4,Cheese Quesadillas,233,2446.50
5,Veggie Burger,238,2499.00
6,Steak Tacos,214,2985.30
7,Cheese Lasagna,207,3208.50
8,Fettuccine Alfredo,249,3610.50
9,Shrimp Scampi,239,4768.05


In [29]:
# What is the Total Revenue Generated by the Top 10 Highest Revenue Orders?
print("\n" + "=" * 60)
print("TOTAL REVENUE GENERATED BY THE TOP 10 HIGHEST REVENUE ORDERS")
print("=" * 60)

pd.read_sql_query("""
SELECT
    ROUND(SUM(order_total), 2) AS top_10_orders_revenue
FROM (
    SELECT order_id, SUM(revenue) AS order_total
    FROM order_analysis
    GROUP BY order_id
    ORDER BY order_total DESC
    LIMIT 10
);
""", conn)



TOTAL REVENUE GENERATED BY THE TOP 10 HIGHEST REVENUE ORDERS


,top_10_orders_revenue
0,1860.95


#### Temporal Trends


In [31]:
# Orders by Hour of Day
print("\n" + "=" * 40)
print("ORDERS BY HOUR OF DAY")
print("=" * 40)

pd.read_sql_query("""
SELECT
    order_hour,
    COUNT(DISTINCT order_id) AS total_orders,
    ROUND(SUM(revenue), 2) AS total_revenue
FROM order_analysis
GROUP BY order_hour
ORDER BY order_hour;
""", conn)


ORDERS BY HOUR OF DAY


,order_hour,total_orders,total_revenue
0,10,2,63.35
1,11,286,8122.50
2,12,644,21718.40
3,13,590,20640.25
4,14,423,12615.70
5,15,354,9803.90
6,16,479,13711.65
7,17,618,17869.50
8,18,595,16861.50
9,19,497,14172.60


- Peak revenue hours:

    - 12–13 (Lunch peak)

    - 17–18 (Dinner peak)

- Orders drop sharply after 21:00.

# Conclusions
This analysis reveals that the restaurant’s performance is driven by a concentrated set of high-value menu items and strong demand during specific time windows. Italian and Asian cuisines form the core of the restaurant’s revenue engine, while a long tail of low-performing items contributes relatively little. By optimizing menu focus, refining pricing and bundling strategies, and aligning operations with peak demand, the restaurant can improve both revenue efficiency and customer experience.

# Recommendations 

The restaurant should prioritize its top-performing items by ensuring consistent availability, prominent menu placement, and potential feature promotions for dishes like Korean Beef Bowl, Spaghetti & Meatballs, and Tofu Pad Thai. These items are proven revenue drivers and should form the backbone of the menu strategy.

Low-revenue, low-frequency items such as Chicken Tacos and Potstickers should be reviewed for possible repositioning, repricing, or removal. Some of these items may benefit from being bundled with high-performing dishes to increase their exposure and contribution without expanding the menu.

Given the strong performance of Italian and Asian categories, future menu development should focus on expanding within these cuisines rather than introducing entirely new categories. Limited-time offerings or seasonal specials within these categories could further increase revenue without increasing operational complexity.

Operational planning should align with observed demand patterns. Staffing, inventory preparation, and marketing efforts should focus on peak lunch and dinner hours, while quieter periods could be leveraged for targeted promotions or cost-control measures.

Finally, frequent low-price items such as Edamame and Fries present an opportunity for strategic upselling. Pairing them with premium mains or offering small price adjustments could significantly increase their revenue impact without affecting order volume.